In [1]:
import numpy as np
import itertools
from sage.symbolic.expression_conversions import polynomial

def calculate_sigma_mgf_explicitly_mod_degree(n, l):
    # Find the eigenvalues of generalised permutations
    eigenvalue_sets = []
    for permutation in Permutations(n):
        permutation_matrix = permutation.to_matrix()
        for coefficients_z_mod_l in itertools.product(range(l), repeat=n):
            coefficients = np.array([CDF(e^(2 * I * pi * c / l)) for c in coefficients_z_mod_l])
            M = matrix(np.diag(coefficients)) * permutation_matrix
            eigenvalue_sets.append(M.eigenvalues())
    
    Sym = SymmetricFunctions(CC)
    m = Sym.m()
    h = Sym.h()
    total = 0
    # Get the average for each partition, up to and including degree n
    for degree in range(n+1):
        for tau in Partitions(degree):
            m_tau = m(tau)
            h_tau_poly = h(tau).expand(n)
            total_per_tau = 0
            for eigenvalue_set in eigenvalue_sets:
                total_per_tau += h_tau_poly(eigenvalue_set)
            avg_per_tau = total_per_tau / len(eigenvalue_sets)
            total += avg_per_tau * m_tau
    return total

In [2]:
# This extends the idea behind the truncate function on MPolynomial, but now truncating based on total degree
def truncate(poly, n):
    R = poly.parent()
    
    return R({key: coef for key, coef in poly.monomial_coefficients().items()
              if sum(key) <= n})

In [3]:
def calculate_sigma_mgf_with_formula_mod_degree(n, l):
    
    # Polynomial ring in n polys
    R = PolynomialRing(CC, n, "x")
    variables = R.gens()

    product = R(1)
    for degree in range(l, n+1, l):
        # TODO: there is probably a more effecient way to do this
        # Consider all possible monomials of this degree
        seen_monomials = set()
        for selected_vars in itertools.combinations_with_replacement(variables, degree):
            monomial = 1
            for var in selected_vars:
                monomial *= var
            if monomial in seen_monomials:
                continue
            # Now create a truncated version of 1/(1-monomial)=1 + monomial + monomial ^ 2 + ...
            monomial_power = 1
            total = 0
            for _ in range(0, n+1, degree):
                total += monomial_power
                monomial_power *= monomial
            product *= total
            seen_monomials.add(monomial)
            product = truncate(product, n)
    
    return product

In [4]:
def symmetric_poly_to_poly(poly, n):
    poly = poly.expand(n)
    return poly

In [5]:
n = 2
l = 2

explicit_result = symmetric_poly_to_poly(calculate_sigma_mgf_explicitly_mod_degree(n, l), n)
print(explicit_result)

formula_result = calculate_sigma_mgf_with_formula_mod_degree(n, l)
print(formula_result)

(1.00000000000000 + 6.93889390390721e-18*I)*x0^2 + (1.00000000000000 + 1.38777878078144e-17*I)*x0*x1 + (1.00000000000000 + 6.93889390390721e-18*I)*x1^2 + (-7.63278329429795e-17 - 8.32667268468867e-17*I)*x0 + (-7.63278329429795e-17 - 8.32667268468867e-17*I)*x1 + 1.00000000000000
x0^2 + x0*x1 + x1^2 + 1.00000000000000


In [6]:
for n in range(1, 7):
    # Note: for some reason, for l=7, sage internals fall into an infinite loop
    # also my code is very inefficient and so starts to run very slowly as n increases, so I will reduce l to combat that
    for l in range(1, max(3, 9 - 2 * n)):
        
        explicit_result = symmetric_poly_to_poly(calculate_sigma_mgf_explicitly_mod_degree(n, l), n)        
        formula_result = truncate(calculate_sigma_mgf_with_formula_mod_degree(n, l), n)
        
        for coef_diff in (explicit_result - formula_result).monomial_coefficients().values():
            assert abs(coef_diff) < 0.000001, f"Difference of coefficient is {coef_diff} for explicit result {explicit_result} and predicted result {formula_result}"

        print(f"n: {n}, l: {l}")

n: 1, l: 1
n: 1, l: 2
n: 1, l: 3
n: 1, l: 4
n: 1, l: 5
n: 1, l: 6
n: 2, l: 1
n: 2, l: 2
n: 2, l: 3
n: 2, l: 4
n: 3, l: 1
n: 3, l: 2
n: 4, l: 1
n: 4, l: 2
n: 5, l: 1
n: 5, l: 2
n: 6, l: 1
n: 6, l: 2
